In [1]:
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from g2p_en import G2p
import re
import torch.nn.utils as utils

import math

import pickle
import Levenshtein
import os

import matplotlib.pyplot as plt
from IPython.display import Audio
import soundfile as sf

In [2]:
import sys, os
sys.path.insert(0, "/home/k2/src/textlesslib")  
import textless
print("textless loaded from:", textless.__file__)

import os, re, pathlib, torch, torchaudio
from textless.vocoders.tacotron2.vocoder import TacotronVocoder

textless loaded from: /home/k2/src/textlesslib/textless/__init__.py


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
vocoder = TacotronVocoder.by_name("hubert-base-ls960", "kmeans", 100).to(device)
srOut = vocoder.output_sample_rate
print("vocoder SR:", srOut, "| device:", device)

/home/k2/src/textlesslib/textless/vocoders/tacotron2/vocoder.py:118: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt_dict = torch.load(tacotron_model_path, map_location=t

vocoder SR: 22050 | device: cuda


In [ ]:
"""
Train GENERAL-VOCAB EMG-to-audio conversion.

For description of the data, please see largeVocabDataVisualization.ipynb

Given a sentence transcription:
    1) Obtain audio via gTTS.
    2) From audio, obtain HuBERT units.
    3) Decode HuBERT units using CTC loss. 
"""

"""https://pypi.org/project/Levenshtein/ - install this Lev distance."""